# Cryptography $CC4017$ -- Extra 11

## Q1: Demonstrating forgeability of plain RSA

A key property of digital signatures is that of existential unforgeability, which is expressed as a security game as follows.

**Existential Unforgeability**:

1 - We generate a keypair$sk, pk$

2 - We give $pk$ to the adversary, and allow him to request signatures of arbitrary messages. This means that the adversary is free to request signatures of anything, but every signed message is recorded.

3 - The adversary must then produce a pair $$m, s$$. He wins the game if 

i.$ $s$ is a valid signature for $m$

ii.$ the signature of $m$ was not requested in step 2.

**Plain RSA signature scheme** 

Plain RSA as a signature scheme sets $$e, n$$ as the public key and $$d, n$$ as the private key. Signatures of m are $m^{d}$ and validation of $$m, s$$ is checking if $m=s^{e}$.

It is easy to see that plain RSA is not existentially unforgeable. Indeed, anyone can produce forgery$1,1$, as $1^{e}= 1$ regardless of the value of $e$. However the issues of signing with plain RSA go aboveand beyond this singular case. Let’s go for a slightly more challenging scenario.

**Question:** 

Describe how an adversary can produce a valid forgery for the following experiment with probability 1.

**Existential unforgeability with a twist**

1 - We generate a keypair $$sk, pk$$ and select a bad value $b$.

2 - We give $pk$ and $b$ to the adversary, and allow him to request signatures of arbitrary messages. This means that the adversary is free to request signatures of anything, but every signed message is recorded.

3 - The adversary must then produce a signatures. He wins the game if 

i.$ $s$ is a valid signature for $b$

ii.$ the signature of $b$ was **not** requested in step 2.



## Answer

>**Exploiting Plain RSA**
>
>In plain RSA, the signature of a message $m$ is $s = m^d \mod n$, and verification involves checking if $m = s^e \mod n$.
>
>Given the properties of RSA, the adversary can exploit the multiplicative property of RSA signatures. Specifically, if the adversary can find two messages $m_1$ and $m_2$ such that $m_1 \cdot m_2 \equiv b \mod n$, then the adversary can use the signatures of $m_1$ and $m_2$ to forge a signature for $b$.
>
>**Request Signatures:**
>
>The adversary requests the signatures for two messages $m_1$ and $m_2$ such that $m_1 \cdot m_2 \equiv b \mod n$.
>
>Let $s_1 = m_1^d \mod n$ and $s_2 = m_2^d \mod n$ be the signatures for $m_1$ and $m_2$, respectively.
>
>**Forge Signature:**
>
>The adversary computes the forged signature $s$ for $b$ as $s = s_1 \cdot s_2 \mod n$.
Verification:
>
>To verify the forged signature $s$, we check if $b = s^e \mod n$.
>
>Since $s = s_1 \cdot s_2 \mod n$, we have: 
>
>$s^e \equiv (s_1 \cdot s_2)^e \mod n \equiv s_1^e \cdot s_2^e \mod n \equiv m_1 \cdot m_2 \mod n \equiv b \mod n $
>
>Therefore, $s$ is a valid signature for $b$.
>
> By exploiting the multiplicative property of RSA, the adversary can produce a valid forgery for the message (b) with probability 1.

## Q2: Shamir Secret Sharing

In the extra work of class 1, we overviewed a technique for secret sharing that allowed messages to be split in parts, such that it could only be reconstructed if a subset of participants agreed to participate i.e. revealed their 'shares' of the original value.

Shamir Secret Sharing is based on polynomial interpolation over finite fields. Consider a point $(0 , 1)$. It is easy to observe that there is an overwhelming number of polynomials p of degree 1 such that $p (0) = 1$.

E.g.

$p ( x ) = x +1; p ( x ) = 2 x +1; p ( x ) = 4 x +1; etc.$

However, when given two points $(0 , 1)$ and $(1 , 3)$ there is only one polynomial p for which these points are valid: $p ( x ) = 2 x +1$.

The intuition for Shamir Secret Sharing is that secrets are represented as polynomials, and shares are represented as points in said polynomial. As such, one can very flexibly select the number of necessary shares and the threshold for secret reconstruction by adjusting the parameters of the system.

We will consider integer polynomial coefficients, and standard integer arithmetic. A practical implementation of shamir secret sharing requires computation over a finite field to ensure privacy. However, for this didatic exercise, we can disregard this very simple adaptation , as the intuitions we want to understand can be observed in integer arithmetic, which is slightly more straightfoward to explore.

## Question - P1:

Implement the secret sharing function that takes value $x$ and produces a set of four shares $x_1$ , $x_2$ , $x_3$ , $x_4$ such that any three can be used to reconstruct the original value.

This will entail responding to the following problems:

- 1- What degree should the polynomial be, so that three points are sufficient to reconstruct the value, but two points can never be enough?
- 2- How can we generate a polynomial f of that degree, such that f (0) = x ?

## Question - P2:

Implement a function for polynomial interpolation, that takes $n$ points, and recovers the only polynomial of degree $n-1$ that contains those points.

Show how this allows for the secret to be recovered:

- 1- Generate a polynomial f for secret number 1001 and shares (points) for that secret.
- 2- Take only the minimum amount of necessary shares (randomly selected) and retrieve the polynomial f '
- 3- Show that, for $x = 0$, $f ' ( x ) = 1001$, and thus the secret can be recovered.

## Question - P3:

Use what you implemented to test the following:

- 1- Generate a polynomial f for secret number 100 and shares (points) for that secret: $x_1$ , $x_2$ , $x_3$ , $x_4$ .
- 2- Generate another polynomial g for secret number 550 and shares (points) for that secret $y_1$ , $y_2$ , $y_3$ , $y_4$.
- 3- Calculate $z_1 = x_1 + y_1$ ; $z_2 = x_2 + y_2$ and $z_3 = x_3 + y_3$
- 4- Use your secret recovery method using points $z_1$ , $z_2$ , $z_3$ . What can you conclude from the result? Justify what happened.

# Answer:

## P1:

#### 1 - What degree should the polynomial be?

To ensure that any three points are sufficient to reconstruct the value, but two points can never be enough, we need the degree of the polynomial $f(x)$ must be 2. This is because:
    - A polynomial of degree d is uniquely determined by d+1 points.
    - Require three points to reconstruct the secre, te degree must be $d = 2$ so that three points are required to reconstruct the polynomial via interpolation.
    - If the degree were 1, only two points would be required.

Thus, we use a quadratic polynomial:

$$f(x) = a_0 + a_1x + a_2x^2$$

where $a_2$, $a_1$, and $a_0$ are integer coefficients. To ensure that $f(0) = x$, we set $a_0 = x$.

#### 2 - How can we generate a polynomial f of that degree, such that $f(0)=x$?

To generate a quadratic polynomial $f(x)$ such that $f(0) = x$, we can choose random integer coefficients $a_1$ and $a_2$ and set $a_0 = x$.  This ensure that at $x = 0$. 

The polynomial will be of the form:

$$f(x) = x + a_1x + a_2x^2$$

To generate the shares, we evaluate the polynomial at four different points $x_1, x_2, x_3, x_4$.

$$(x_1,x_2,x_3,x_4) = (f(1), f(2), f(3), f(4))$$

In [3]:
import random

def shamir_secret_sharing(secret, num_shares = 4, threshold = 3):
    """
    Parameters:
    secret (int): The secret to be shared.
    num_shares (int): The number of shares to generate.
    threshold (int): The minimum number of shares required to reconstruct the secret.
    """

    if threshold > num_shares:
        raise ValueError("The threshold must be less than or equal to the number of shares.")

    # Generate random coefficients for the polynomial
    a0 = secret
    a1 = random.randint(1, 100)
    a2 = random.randint(1, 100)


    def f(x):
        return a2 * x**2 + a1 * x + a0

    shares = [(i, f(i)) for i in range(1, num_shares + 1)]
    return shares

# Example usage
secret = 42 
shares = shamir_secret_sharing(secret)
print("Secret:", secret)
print("Shares:", shares)

Secret: 42
Shares: [(1, 169), (2, 384), (3, 687), (4, 1078)]


## P2:

In [77]:
from functools import reduce
from decimal import Decimal, getcontext

getcontext().prec = 50

def lagrange_interpolation(x, points):
    def basis(j):
        p = [Decimal(x - points[m][0]) / Decimal(points[j][0] - points[m][0]) for m in range(len(points)) if m != j]
        return reduce(lambda x, y: x*y, p)
    
    return sum(Decimal(points[j][1]) * basis(j) for j in range(len(points)))

# Example usage
secret = 1001
shares = shamir_secret_sharing(secret)
print("Secret:", secret)
print("Shares:", shares)

selected_shares = random.sample(shares, 3)
print("Selected shares:", selected_shares)

recovered_secret = lagrange_interpolation(0, selected_shares)
recovered_secret = int(round(recovered_secret))
print("Recovered secret:", recovered_secret)


Secret: 1001
Shares: [(1, 1102), (2, 1395), (3, 1880), (4, 2557)]
Selected shares: [(1, 1102), (3, 1880), (2, 1395)]
Recovered secret: 1001


In [78]:
## P3:

# Generate shares for secret 100
secret1 = 100
shares1 = shamir_secret_sharing(secret1)
print("Shares for secret 100:", shares1)

# Generate shares for secret 550
secret2 = 550
shares2 = shamir_secret_sharing(secret2)
print("Shares for secret 550:", shares2)

# Calculate combined shares
z1 = (shares1[0][0], shares1[0][1] + shares2[0][1])
z2 = (shares1[1][0], shares1[1][1] + shares2[1][1])
z3 = (shares1[2][0], shares1[2][1] + shares2[2][1])
combined_shares = [z1, z2, z3]
print("Combined shares:", combined_shares)

# Recover the secret
recovered_secret = lagrange_interpolation(0, combined_shares)
recovered_secret = int(round(recovered_secret))
print("Recovered secret:", recovered_secret)


Shares for secret 100: [(1, 189), (2, 434), (3, 835), (4, 1392)]
Shares for secret 550: [(1, 714), (2, 1050), (3, 1558), (4, 2238)]
Combined shares: [(1, 903), (2, 1484), (3, 2393)]
Recovered secret: 650


### Conclusion:
When we combine the shares of two secrets and recover the secret from the combined shares, the result should be the sum of the two original secrets. This is because the addition of the shares corresponds to the addition of the polynomials, and evaluating the combined polynomial at \( x = 0 \) will give the sum of the original secrets.

In this case, the recovered secret should be \( 100 + 550 = 650 \). This demonstrates that the secret sharing scheme is linear, meaning that the sum of the shares corresponds to the sum of the secrets.